# CSME2 Bonus Point Assignment 1

<div style="text-align: right;font-size: 0.8em">Document Version 1.0.0, released 01/12/2021</div>
For detailed task instructions, please refer to the assignment PDF.

DO NOT CLEAR THE OUTPUT of the notebook you are submitting!

In [1]:
# Add any additional import you need in this cell
import numpy as np
from sklearn.cluster import kmeans_plusplus, KMeans
from scipy.io import loadmat
np.random.seed(1234)

### Setup
__Task A.1__ Load the data into the following numpy arrays. For the output, only use the first torque variable.
Randomly split the data from the file ```sarcos_inv.mat``` into a training set (80%) and a validation set (20%).

In [219]:
# postion/velocity/acceleration  = 7 + 7 + 7 = 21
# The last 7 columns reprensent the torche corresponding each joint

train_data = loadmat("./sarcos_inv.mat")["sarcos_inv"]
test_data = loadmat("./sarcos_inv_test.mat")["sarcos_inv_test"]

# Shuffle data so that it can be random splitted
perm = np.random.RandomState(42).permutation(len(train_data))

# Input and output training data

# Split train_data as 80% training data and 20% validation data
n = train_data.shape[0]
xs_train = train_data[perm[:int(n*0.8)], :21]  
ys_train = train_data[perm[:int(n*0.8)], 22].reshape((-1,1))

# Input and output validation data
xs_valid = train_data[int(n*0.8):, :21]  
ys_valid = train_data[int(n*0.8):, 22].reshape((-1,1))

# Input and output test data
xs_test = test_data[:, :21]
ys_test = test_data[:, 22].reshape((-1,1))

In [220]:
# Check for yourself 
# The following should lead to output 
# (35587, 21)
# (35587, 1)
# (8897, 21)
# (8897, 1)
# (4449, 21)
# (4449, 1)

print(xs_train.shape)
print(ys_train.shape)
print(xs_valid.shape)
print(ys_valid.shape)
print(xs_test.shape)
print(ys_test.shape)

(35587, 21)
(35587, 1)
(8897, 21)
(8897, 1)
(4449, 21)
(4449, 1)


__Task A.2__ Standardize the data such that
1. Training inputs have mean 0
2. Each training input variable has variance 1
3. The training outputs have mean 0
4. Apply the same transformation to the validation and test data

Implement this manually, i.e., do not use a ready scaler like the one provided by scikit-learn.

In [221]:
# function to standardize data
def standardize(data):
    N, D = data.shape
    data_std = data.copy()
    avg = np.mean(data_std, axis=0)
    std = np.std(data_std, axis=0)
    
    data_std = (data_std - avg)/std
    
    return data_std          

# Store the standardized data in the following variables
xs_train_std = standardize(xs_train)
ys_train_std = standardize(ys_train)

xs_valid_std = standardize(xs_valid)
ys_valid_std = standardize(ys_valid)

xs_test_std = standardize(xs_test)
ys_test_std = standardize(xs_test)

In [222]:
# Check for yourself
# The following should lead to (roughly) six zeros and three arrays with (approximately) ones
print(np.mean(xs_train_std))
print(np.mean(ys_train_std))
print(np.mean(xs_valid_std))
print(np.mean(ys_valid_std))
print(np.mean(xs_test_std))
print(np.mean(ys_test_std))

print(np.var(xs_train_std, axis=0))
print(np.var(xs_valid_std, axis=0))
print(np.var(xs_test_std, axis=0))

3.465118419636068e-15
9.484019430860921e-17
-5.065607583254138e-16
6.069601879034238e-17
4.865783026033802e-16
4.865783026033802e-16
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


__Task A.3__

In [223]:
# Task A.3.a
# Implement a function estimating the variance
def my_variance(xs):
    """Calculate the empirical variance of a given vector of scalars
    
    Arguments
    xs      1d numpy array
    
    Returns
    The empirical variance of the provided vector
    """
    # Your implementation
    return float(sum(np.square(xs-xs.mean()))/(len(xs)-1))

In [224]:
# Task A.3.b
# Calculate the variance of ys_train_std using your function my_variance
var_ys_train = my_variance(ys_train_std)

In [225]:
# Task A.3.c
# Implement a function calculating the SMSE between two 1d numpy arrays given a normalizing factor
def my_smse(z1, z2, s):
    """Calculate the Standardized Mean Squared Error (SMSE)
    
    Arguments
    z1      1d numpy array (usually the predictions)
    z2      1d numpy array (usually the test data)
    s       Normalization factor (usually the variance of the test data)
    
    Returns
    The SMSE of the provided data
    """
    # Your implementation
    N = len(z1)
    MSE = float(sum(np.square(z2-z1)))
    return MSE/(N*np.square(s))

## Linear regression
### Simple linear regression
__Task A.4__

In [226]:
# This variable should contain the weights corresponding to simple linear regression (LS criterion, no bias term)

# Using least square method
w_lr = np.linalg.inv(xs_train_std.T @ xs_train_std) @ xs_train_std.T @ ys_train_std

# This variable should contain the predictions using w_lr on the validation data
ys_pred_valid = xs_valid_std @ w_lr

# This should contain the resulting smse
smse_lr = my_smse(ys_pred_valid, ys_valid_std, var_ys_train)
smse_lr

0.07566694209945357

### Linear regression with polynomial features
__Task A.5__

In [278]:
# Test with library
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
X = np.arange(6).reshape(2, 3)
poly = PolynomialFeatures(2)
print(poly.fit_transform(X))
poly.fit_transform(X).shape

[[ 1.  0.  1.  2.  0.  0.  0.  1.  2.  4.]
 [ 1.  3.  4.  5.  9. 12. 15. 16. 20. 25.]]


(2, 10)

In [285]:
X = np.arange(6).reshape(2, 3)
N, D = X.shape
degree = 2

    

In [ ]:
# Task A.5.a
def my_poly_features(xs, degree):
    """Generates polynomial features from given data
    
    The polynomial features should include monomials (i.e., x_i, x_i**2 etc)
    and interaction terms (x_1*x_2 etc), but no repetitions.
    The order of the samples should not be changed through the transformation.
    
    Arguments
    xs      2d numpy array of shape (N,D) containing N samples of dimension D
    degree  Maximum degree of polynomials to be considered
    
    Returns
    An (N,M) numpy array containing the transformed input
    """
    # Your implementation
    N, D = xs.shape
    
    
    pass

In [ ]:
# Tasks A.5.b
# This variable should contain the weights corresponding to linear regression using polynomial features up to degree 2 and 3
# w_poly2 =
# w_poly3

# This variable should contain the predictions using w_poly2 and w_poly3 on the validation data
# ys_pred_poly2_valid = 
# ys_pred_poly3_valid = 

# This should contain the resulting smse
# smse_poly2 = my_smse(ys_pred_poly2_valid, ys_valid_std, var_ys_train)
# smse_poly3 = my_smse(ys_pred_poly3_valid, ys_valid_std, var_ys_train)

## Clustering
__Task B.1__ Implement the basic $K$-Means algorithm.

In [ ]:
def my_kmeans(xs, init_centers, n_iter):
    """Runs the K-Means algorithm from a given initialization
    
    Arguments
    xs            2d numpy array of shape (N,D) containing N samples of dimension D
    init_centers  2d numpy array of shape (K,D) containing the initial cluster centers
    n_iter        Number of iterations of the K-Means algorithm
    
    Returns
    An (K,D) numpy array containing the final cluster centers
    """
    # Your implementation
    pass

__Task B.2__ Generate test data set and plot it.

In [ ]:
# Test data of shape (100,2)
# xs_cluster_test = 

__Task B.3__ Run your $K$-Means algorithm on the test data for $K=2,3,4,5$ clusters and plot the final cluster centers.

In [ ]:
# Use kmeans_plusplus(xs_cluster_test, K, random_state=0) for initialization

## Radial Basis Function Network
__Task C.1__ Find $K=100$ cluster centers using $K$-Means.

In [ ]:
# This 100x21 numpy array should contain the cluster centers
# xs_centers = 

__Task C.2__ Implement the Gaussian basis functions and transform the data accordingly

In [ ]:
# xs_train_gauss = 

__Task C.3__ Run simple linear regression on the transformed data and evaluate it on the test set

In [ ]:
# This should contain the resulting predictions on the validation data set
# ys_pred_gauss_valid = 

# This should contain the corresponding SMSE
# smse_gauss = 

__Open task__ Can you improve the performance of the RBF network?

__Task C.4__ Evaluate your final model (either the one from Task C.3 or your improved model from the open task) on the test data

In [ ]:
# This should contain the predictions on the test data set
# ys_pred_test = 

# This should contain the resulting SMSE on the test data
# smse_test = 